In [ ]:
import os
import re
import json
import requests
from bs4 import BeautifulSoup
import time



In [ ]:
#provide url for fixer uppers here:
urls=['https://newyork.craigslist.org/search/ossining-ny/rea?bundleDuplicates=1&hasPic=1&lat=41.1673&lon=-73.8379&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://cedarrapids.craigslist.org/search/middle-amana-ia/rea?bundleDuplicates=1&hasPic=1&lat=41.8024&lon=-91.9241&query=fixer%20upper&search_distance=550#search=1~gallery~0~0',
      'https://siouxfalls.craigslist.org/search/lake-park-ia/rea?bundleDuplicates=1&hasPic=1&lat=43.4313&lon=-95.3152&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://oklahomacity.craigslist.org/search/cromwell-ok/rea?bundleDuplicates=1&hasPic=1&lat=35.3627&lon=-96.3548&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://shoals.craigslist.org/search/russellville-al/rea?bundleDuplicates=1&hasPic=1&lat=34.4789&lon=-87.6851&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://littlerock.craigslist.org/search/story-ar/rea?bundleDuplicates=1&hasPic=1&lat=34.5177&lon=-93.4784&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://amarillo.craigslist.org/search/white-deer-tx/rea?bundleDuplicates=1&hasPic=1&lat=35.3671&lon=-101.2498&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://fortdodge.craigslist.org/search/humboldt-ia/rea?bundleDuplicates=1&hasPic=1&lat=42.7087&lon=-94.232&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://wyoming.craigslist.org/search/powder-river-wy/rea?bundleDuplicates=1&hasPic=1&lat=43.3491&lon=-106.8546&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://boise.craigslist.org/search/idaho-city-id/rea?bundleDuplicates=1&hasPic=1&lat=43.9336&lon=-115.7391&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://nashville.craigslist.org/search/dickson-tn/rea?bundleDuplicates=1&hasPic=1&lat=36.0844&lon=-87.4926&query=fixer%20upper&search_distance=390#search=1~gallery~0~0',
      'https://littlerock.craigslist.org/search/hot-springs-national-park-ar/rea?bundleDuplicates=1&hasPic=1&lat=34.5295&lon=-93.0207&query=fixer%20upper&search_distance=370#search=1~gallery~0~0',
      'https://monroe.craigslist.org/search/marion-la/rea?bundleDuplicates=1&hasPic=1&lat=32.9349&lon=-92.2852&query=fixer%20upper&search_distance=290#search=1~gallery~0~0',
      'https://dallas.craigslist.org/search/italy-tx/rea?bundleDuplicates=1&hasPic=1&lat=32.1198&lon=-96.8774&query=fixer%20upper&search_distance=290#search=1~gallery~0~0',
      'https://nacogdoches.craigslist.org/search/garrison-tx/rea?bundleDuplicates=1&hasPic=1&lat=31.8199&lon=-94.5245&query=fixer%20upper&search_distance=220#search=1~gallery~0~0',
      'https://collegestation.craigslist.org/search/burlington-tx/rea?bundleDuplicates=1&hasPic=1&lat=31.0026&lon=-96.8596&query=fixer%20upper&search_distance=168#search=1~gallery~0~0',
      'https://westslope.craigslist.org/search/hanksville-ut/rea?bundleDuplicates=1&hasPic=1&lat=37.9362&lon=-110.4813&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://eastky.craigslist.org/search/pinsonfork-ky/rea?bundleDuplicates=1&hasPic=1&lat=37.5264&lon=-82.2216&query=fixer%20upper&search_distance=590#search=1~gallery~0~0',
      'https://westslope.craigslist.org/search/torrey-ut/rea?bundleDuplicates=1&hasPic=1&lat=38.2037&lon=-110.918&query=fixer%20upper&search_distance=550#search=1~gallery~0~0',
      'https://bozeman.craigslist.org/search/yellowstone-national-park-wy/rea?bundleDuplicates=1&hasPic=1&lat=44.6363&lon=-110.0213&query=fixer%20upper&search_distance=690#search=1~gallery~0~0']







In [ ]:
def images_download(url, folder_name_here):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"
    }

    # Get the folder name
    folder_name = folder_name_here+'/' + re.search(r'(\d+).html$', url).group(1)
    
    # Check if directory exists
    if os.path.exists(folder_name):
        print("Duplicate exists for ", re.search(r'(\d+).html$', url).group(1))
        return

    # If the directory doesn't exist, proceed with the scraping and downloading
    os.makedirs(folder_name)

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extracting the JavaScript array using regular expression
        script_content = soup.find('script', text=re.compile('var imgList = ')).string
        img_list_str = re.search(r'var imgList = (\[.*?\]);', script_content).group(1)

        # Parsing the extracted JSON string
        img_list = json.loads(img_list_str)

        # Getting all image URLs from the parsed JSON list
        img_urls = [img['url'] for img in img_list]

        # Download and save each image
        for idx, img_url in enumerate(img_urls, 1):
            img_response = requests.get(img_url, headers=headers)
            img_response.raise_for_status()  # Raise an error for bad responses
            with open(f'{folder_name}/image_{idx}.jpg', 'wb') as file:
                file.write(img_response.content)
            time.sleep(0.1)  # Sleep for a short while between image downloads
        print("Images downloaded for: ", re.search(r'(\d+).html$', url).group(1))
    except requests.RequestException as e:
        print(f"Error encountered: {e}")
    except Exception as e:
        print(f"Unexpected error encountered: {e}")


In [20]:
def getting_all_house_links(urls:list):  #the urls are the list of urls we want to scrape. this is based on keywords such as fixer upper or renovated
  def individual_url_getter(url):  #from the list of urls, each url is passed here. this function then gets house urls within the keyword based url.
  
    URL=url

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"
    }

    response = requests.get(URL, headers=headers)

    soup = BeautifulSoup(response.content, "html.parser")

    # Assuming your soup object is named 'soup'
    links = soup.select('li.cl-static-search-result a')

    # Extract the href attribute from each link
    urls = [link['href'] for link in links]

    return urls

  all_urls=[]   #this is where all the house urls are saved
  for idx, url in enumerate(urls):
    print(idx)
    url=url
    print(url)
    print('number of houses detected ', len(all_urls))
    all_urls.extend(individual_url_getter(url))

  all_urls=list(set(all_urls))
  print('the number of unique housings by id are', len(all_urls))
  return list(set(all_urls))

In [ ]:
getting_all_house_links(urls)

In [28]:


#start downloading images
def scrape(target_older: str, urls: list):
  all_urls=getting_all_house_links(urls)
  
  i=0
  for url in all_urls:

    time.sleep(1)
    images_download(url, target_older)
    i+=1
    print('images downloaded for ' + str(i) + ' houses')

In [ ]:
scrape('fixer_uppers', urls)

# Starting the process for renovated now

In [29]:
#renovated URL here
renovated_urls=['https://bgky.craigslist.org/search/woodbury-ky/rea?bundleDuplicates=1&hasPic=1&lat=37.089&lon=-86.6147&min_price=100000&query=renovated&search_distance=810#search=1~gallery~0~0',
      'https://nwks.craigslist.org/search/wilsonville-ne/rea?bundleDuplicates=1&hasPic=1&lat=40.0565&lon=-100.0696&min_price=100000&query=renovated&search_distance=810#search=1~gallery~0~0',
      'https://scottsbluff.craigslist.org/search/crook-co/rea?bundleDuplicates=1&hasPic=1&lat=40.9897&lon=-102.795&min_price=100000&query=renovated&search_distance=630#search=1~gallery~0~0',
      'https://elko.craigslist.org/search/montello-nv/rea?bundleDuplicates=1&hasPic=1&lat=41.5994&lon=-113.9402&min_price=100000&query=renovated&search_distance=630#search=1~gallery~0~0',
      'https://columbiamo.craigslist.org/search/arrow-rock-mo/rea?bundleDuplicates=1&hasPic=1&lat=39.1383&lon=-92.8717&query=renovated&search_distance=600#search=1~gallery~0~0',
      'https://boston.craigslist.org/search/boston-ma/rea?lat=42.3583&lon=-71.0603&min_price=100000&query=renovated&search_distance=1000#search=1~gallery~0~0',
      'https://tulsa.craigslist.org/search/quinton-ok/rea?bundleDuplicates=1&hasPic=1&lat=35.1979&lon=-95.5471&query=renovated&search_distance=600#search=1~gallery~0~0',
      'https://evansville.craigslist.org/search/wheatland-in/rea?bundleDuplicates=1&hasPic=1&lat=38.6386&lon=-87.2612&query=renovated&search_distance=580#search=1~gallery~0~0',
      'https://littlerock.craigslist.org/search/gurdon-ar/rea?bundleDuplicates=1&hasPic=1&lat=33.951&lon=-92.9979&query=renovated&search_distance=330#search=1~gallery~0~0',
      'https://tuscaloosa.craigslist.org/search/bankston-al/rea?bundleDuplicates=1&hasPic=1&lat=33.7833&lon=-87.647&query=renovated&search_distance=330#search=1~gallery~0~0',
      'https://atlanta.craigslist.org/search/roopville-ga/rea?bundleDuplicates=1&hasPic=1&lat=33.4681&lon=-85.1735&query=renovated&search_distance=330#search=1~gallery~0~0',
      'https://jacksonville.craigslist.org/search/saint-george-ga/rea?bundleDuplicates=1&hasPic=1&lat=30.7101&lon=-82.2709&query=renovated&search_distance=330#search=1~gallery~0~0',
      'https://treasure.craigslist.org/search/okeechobee-fl/rea?bundleDuplicates=1&hasPic=1&lat=27.0004&lon=-80.8594&query=renovated&search_distance=152#search=1~gallery~0~0'

      ]


# sample_url='https://boston.craigslist.org/search/boston-ma/rea?lat=42.3583&lon=-71.0603&min_price=100000&query=renovated&search_distance=1000#search=1~gallery~0~0'



# image_url_getter(sample_url)



In [ ]:
scrape('renovated_new', renovated_urls)